# CIFAR 10 Classifier: Pytorch Lightning Edition

We use Pytorch Lightning in this notebook, it abstracts away all the boiler plate code that we need to add to every pytorch training workflow.

This notebook performs CIFAR10 classification on GPU using:
- Resnet 50 as backbone
- Minimal augmentation
- Experiment tracking using Comet ML

Most of the content is similar to the [Tensorflow](tf-resnet50.ipynb) and [Pytorch](pytorch-resnet50.ipynb) version of the notebook. 

---

@date: 03-Sep-2020 | @author: katnoria

# 1. Imports & Setup

In [1]:
import os
from time import time
from datetime import datetime
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets, models, utils
from torch.utils.data import DataLoader, random_split

# load from .env
from pathlib import Path
from dotenv import load_dotenv


import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.metrics.functional import accuracy

from pytorch_lightning.loggers import CometLogger

In [2]:
pl.seed_everything(42)

42

In [3]:
# Load secrets
%load_ext dotenv
%dotenv ../../.env

In [4]:
COMET_ML_API_KEY = os.getenv("COMET_ML_API_KEY")
len(COMET_ML_API_KEY)

25

In [5]:
def version_info(cls):
    print(f"{cls.__name__}: {cls.__version__}")

In [6]:
# Print version info
version_info(torch)
version_info(pl)

torch: 1.6.0
pytorch_lightning: 0.9.0


# 2. Dataset

We will load CIFAR10 dataset from pytorch datasets

In [7]:
# Hyper params
BATCH_SIZE=128
NUM_WORKERS=12

In [8]:
tfms = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.247, 0.243, 0.261])
])

In [9]:
# Load the dataset
train_ds = datasets.CIFAR10(
    root="./data", train=True, 
    download=True, transform=tfms
)
# Create train and validation splits
train, val = random_split(train_ds, [45000, 5000])
# Create data loaders
train_loader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader = DataLoader(val, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

Files already downloaded and verified


In [10]:
len(train_loader.dataset), len(val_loader.dataset)

(45000, 5000)

In [11]:
test_ds = datasets.CIFAR10(
    root="./data", 
    train=False,
    download=True,
    transform=tfms
)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
len(test_loader.dataset)

Files already downloaded and verified


10000

# 3. Build Model

We will use imagenet pre-trained ResNet50 model. You can swap out the base model with others such as ResNet 18 or ResNet 110. Just make sure the input features of the final linear layer matches with the out features of your base model.

In [12]:
class CIFARTenLitModelV2(pl.LightningModule):
    """CIFAR10 Model"""
    def __init__(self, backbone, learning_rate):
        super().__init__()
        self.learning_rate = learning_rate
        self.backbone = backbone
        self.backbone.fc = nn.Linear(2048, 256)
        self.fc1 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.backbone(x)
        x = F.relu(x)
        out = self.fc1(x)
        return out
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        acc = accuracy(y_hat, y)        
        result = pl.TrainResult(loss)
#         result.log("train_loss", loss, prog_bar=True)
        result.log("train_acc", acc, prog_bar=True)        
        return result
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        acc = accuracy(y_hat, y)        
        result = pl.EvalResult(checkpoint_on=loss)
#         result.log("val_loss", loss)
        result.log("val_acc", acc)
        return result
    
    def test_step(self, batch, batch_index):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        acc = accuracy(y_hat, y)        
        result = pl.EvalResult(checkpoint_on=loss)
        result.log("test_acc", acc)
        return result
        
    
    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.learning_rate)

# 3. Train

## 3.1: Track Experiments
You can track your experiments using tensorboard, w&b or any other tool. I am using comet in this notebook. I quite like the overall experience with [comet ml](https://www.comet.ml/).

In [13]:
comet_logger = CometLogger(api_key=COMET_ML_API_KEY,save_dir='.',project_name="cf10-pl", workspace="katnoria")
# add a tag
comet_logger.experiment.add_tag("R50+Dense")

CometLogger will be initialized in online mode
COMET INFO: old comet version (3.2.0) detected. current: 3.2.1 please update your comet lib with command: `pip install --no-cache-dir --upgrade comet_ml`
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/katnoria/cf10-pl/f775670088834ce7bac4209effdbffbe



Load the pre-trained Resnet 50 model from pytorch 

In [14]:
backbone = models.resnet50(pretrained=True)
for param in backbone.parameters():
    param.requires_grad = False

In [15]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    strict=False,
    verbose=False,
    mode='min'
)

With Pytorch Lightning, the trainer takes care of setting up the training, validation and test loop. It also provides other goodies such as:
* fast dev test (a training loop to check model for errors)
* metric logging
* switching between accelerators (GPU/TPU) without any code change 💯

In [16]:
EPOCHS=100
learning_rate=1e-3

In [17]:
trainer = pl.Trainer(
    fast_dev_run=False, 
    gpus=1, 
    early_stop_callback=early_stop, 
    max_epochs=EPOCHS,
    logger=comet_logger
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [18]:
model = CIFARTenLitModelV2(backbone, learning_rate)

In [19]:
start = time()
print(f"start: {datetime.fromtimestamp(start)}")
# train
trainer.fit(model, train_loader, val_dataloaders=val_loader)
stop = time()


  | Name     | Type   | Params
------------------------------------
0 | backbone | ResNet | 24 M  
1 | fc1      | Linear | 2 K   


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Saving latest checkpoint..
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/katnoria/cf10-pl/f775670088834ce7bac4209effdbffbe
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     epoch [800]     : (0, 99)
COMET INFO:     train_acc [700] : (0.6484375, 1.0)
COMET INFO:     val_acc [100]   : (0.762890636920929, 0.8353515863418579)
COMET INFO:   Uploads:
COMET INFO:     code                     : 1 (4 KB)
COMET INFO:     environment details      : 1
COMET INFO:     filename                 : 1
COMET INFO:     git metadata             : 1
COMET INFO:     git-patch (uncompressed) : 1 (1 KB)
COMET INFO:     installed packages       : 1
COMET INFO:     notebook                 : 1
COMET INFO:     os packages              : 1
COMET INFO: ---------------------------


COMET INFO: Uploading stats to Comet before program termination (may take several seconds)


1

In [ ]:
took = stop - start
print(f"Total training time: {took//60 : .0f}m {took%60:.0f}s")

In [ ]:
trainer.test(model, test_dataloaders=test_loader)

In [20]:
hyper_params = {
    "batch_size": BATCH_SIZE,
    "num_workers": NUM_WORKERS,
    "learning_rate": learning_rate,
    "num_epochs": EPOCHS,
}
comet_logger.experiment.log_parameters(hyper_params)

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/katnoria/cf10-pl/f775670088834ce7bac4209effdbffbe



In [21]:
comet_logger.experiment.end()

COMET INFO: -----------------------------------
COMET INFO: Comet.ml ExistingExperiment Summary
COMET INFO: -----------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/katnoria/cf10-pl/f775670088834ce7bac4209effdbffbe
COMET INFO:   Parameters:
COMET INFO:     batch_size    : 128
COMET INFO:     learning_rate : 0.001
COMET INFO:     num_epochs    : 100
COMET INFO:     num_workers   : 12
COMET INFO: -----------------------------------
COMET INFO: Uploading stats to Comet before program termination (may take several seconds)


# Next Steps

This is a very simple example of training CIFAR10 classifier using a pre-trained network. Its your turn to turn the knobs and see if you can get model to generalise better. Some ideas:

- make the model overfit your training data
- regularize the model to generalize better
- increase/decrease model capacity based on what you find in above steps
- add image augmentation
- use hyperparameter tuning library to find the best set of combination
- rollout your own model from scratch, you can use the tuning library to help design the network too
